In [2]:
from Bio import Entrez
import numpy as np

def search(query):
    Entrez.email = 'your.email@example.com'
    handle = Entrez.esearch(db='pubmed', 
                            sort='most recent', 
                            retmax='500',
                            retmode='xml', 
                            reldate = 7, #only within n days from now
                            term=query)
    results = Entrez.read(handle)
    return results

def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'your.email@example.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

search_results = search('(Biomech*[Title/Abstract] OR locomot*[Title/Abstract])')
id_list = search_results['IdList']
papers = fetch_details(id_list)
print("")
titles = [0 for i in enumerate(papers['PubmedArticle'])]

for i, paper in enumerate(papers['PubmedArticle']):
    titles[i] = papers['PubmedArticle'][i]['MedlineCitation']['Article']['ArticleTitle']

p_num = 0
for i, paper in enumerate(papers['PubmedArticle']):
    print("* [%s](https://www.ncbi.nlm.nih.gov/pubmed/%s)" % (paper['MedlineCitation']['Article']['ArticleTitle'],paper['MedlineCitation']['PMID']))
    for auth in np.arange(1,np.size(paper['MedlineCitation']['Article']['AuthorList'])):
        try:
            print("%s %s," % (paper['MedlineCitation']['Article']['AuthorList'][auth]['LastName'],paper['MedlineCitation']['Article']['AuthorList'][auth]['Initials']))
        except:
            print('NAME ERROR')
    print('%s' % (paper['MedlineCitation']['Article']['Journal']['Title']) )
    p_num = p_num +1
    # #store keywords 
    # print("----")
    # if paper['MedlineCitation']['KeywordList'] != []:
    #     for kw in np.arange(1,np.size(paper['MedlineCitation']['KeywordList'][0])):
    #         print(paper['MedlineCitation']['KeywordList'][0][kw-1][:])
    # else:
    #     print("NO_KEYWORDS")
    print("<br>  ") #linebreak for github md 
    #end keywords test
    print("")




* [Self-supervised learning of the biologically-inspired obstacle avoidance of hexapod walking robot.](https://www.ncbi.nlm.nih.gov/pubmed/30995613)
Faigl J,
Bioinspiration & biomimetics
<br>  

* [Why ruminating ungulates chew sloppily: Biomechanics discern a phylogenetic pattern.](https://www.ncbi.nlm.nih.gov/pubmed/30995252)
Winkler DE,
Fortuny J,
Kaiser TM,
Marcé-Nogué J,
PloS one
<br>  

* [[Archwire segmentation: a solution to the problems involved in treating atypical cases].](https://www.ncbi.nlm.nih.gov/pubmed/30994450)
Bourgoin G,
L' Orthodontie francaise
<br>  

* [Kinematics and electromyographic activity of horse riders during various cross-country jumps in equestrian.](https://www.ncbi.nlm.nih.gov/pubmed/30994050)
Laroche D,
Babault N,
Sports biomechanics
<br>  

* [Neuronal Activity in Non-LNv Clock Cells Is Required to Produce Free-Running Rest:Activity Rhythms in Drosophila.](https://www.ncbi.nlm.nih.gov/pubmed/30994046)
Spontak KR,
Kleeman B,
Cavanaugh DJ,
Journal of

In [3]:
print(np.size(titles,0)) #number of papers. limited to 500. (retmax)

162


In [15]:
import csv
with open('literature_update.csv', 'w', newline = '', encoding = 'utf-8') as myfile:
# myfile =  open('literature_update.csv', 'w',newline ='', encoding='utf-8')
    wr = csv.writer(myfile)
    for item in titles:
        wr.writerow([item])
    